In [2]:
# Artificial Neural Network

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Importing the dataset
train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')

In [3]:
dataset = train
dataset.loc[:, 'pickup_datetime'] = pd.to_datetime(dataset['pickup_datetime'])
dataset.loc[:, 'month'] = dataset.pickup_datetime.dt.month
dataset.loc[:, 'hour'] = dataset.pickup_datetime.dt.hour
dataset.loc[:, 'weekday'] = dataset.pickup_datetime.dt.weekday
dataset.drop(['dropoff_datetime'], axis = 1, inplace=True)
dataset.loc[:, 'quarter'] = np.ceil(dataset.pickup_datetime.dt.minute/15)
dataset.loc[:, 'day'] = dataset.pickup_datetime.dt.day

# convert some of the numerical data to categories
dataset.loc[:, 'store_and_fwd_flag'] = dataset['store_and_fwd_flag'].replace(to_replace = ['N', 'Y'], value = [0,1])

In [4]:
dataset.head(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,month,hour,weekday,quarter,day
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,0,455,3,17,0,2.0,14
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,0,663,6,0,6,3.0,12
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,1,11,1,3.0,19
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,0,429,4,19,2,3.0,6
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,0,435,3,13,5,2.0,26


In [5]:
dataset.loc[:, 'passenger_count'] = dataset['passenger_count'].replace(to_replace= [0,1,2,3,4,5,6,7,8,9], value = ['a', 'b', 'c' \
                                                                                                                  , 'd', 'e', 'f', 'g',\
                                                                                                                  'h', 'i', 'j'])

In [6]:
import math
dataset.loc[:, 'manhattan_dist'] = (abs(dataset['pickup_latitude'].values - dataset['dropoff_latitude'].values) + \
abs(dataset['pickup_longitude'] - dataset['dropoff_longitude']))*6371*math.pi/180.0
dataset.loc[:, 'manhattan_speed'] = 3600*dataset['manhattan_dist'].values/dataset['trip_duration'].values


In [7]:
# direction of movement
import math
dataset.loc[:, 'direction'] = np.arctan((dataset['dropoff_latitude'].values-dataset['pickup_latitude'].values)/ \
(dataset['dropoff_longitude'].values - dataset['pickup_latitude'].values))*180/math.pi

In [7]:
dataset.loc[:, 'pickup_long_round'] = np.around(dataset['pickup_longitude'].values, decimals=3)
dataset.loc[:, 'pickup_lat_round'] = np.around(dataset['pickup_latitude'].values, decimals=3)
dataset.loc[:, 'dropoff_long_round'] = np.around(dataset['dropoff_longitude'].values, decimals=3)
dataset.loc[:, 'dropoff_lat_round'] = np.around(dataset['dropoff_latitude'].values, decimals=3)

In [10]:
from sklearn.cluster import MiniBatchKMeans
sample_ind = np.random.permutation(len(train))[:500000]
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=1000).fit(np.vstack((train[['pickup_longitude', 'pickup_latitude']].values,\
                                                                       train[['dropoff_longitude', 'dropoff_latitude']].values,\
                                                                        validate[['pickup_longitude', 'pickup_latitude']].values,\
                                                                                  validate[['dropoff_longitude', 'dropoff_latitude']].values))[sample_ind])
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['pickup_longitude', 'pickup_latitude']])
train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['dropoff_longitude', 'dropoff_latitude']])

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
validate.loc[:, 'pickup_cluster'] = kmeans.predict(validate[['pickup_longitude', 'pickup_latitude']])
validate.loc[:, 'dropoff_cluster'] = kmeans.predict(validate[['dropoff_longitude', 'dropoff_latitude']])

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(dataset, test_size = 0.2, random_state = 0)

In [12]:
dataset = train
dataset = dataset[dataset['manhattan_dist'] < dataset['manhattan_dist'].quantile(0.999) ]
dataset = dataset[dataset['manhattan_speed'] < dataset['manhattan_speed'].quantile(0.999) ]

# dataset = dataset[dataset['pickup_longitude'] > dataset['pickup_longitude'].quantile(0.001) ]
# dataset = dataset[dataset['pickup_longitude'] < dataset['pickup_longitude'].quantile(0.999) ]
# dataset = dataset[dataset['pickup_latitude'] > dataset['pickup_latitude'].quantile(0.001) ]
# dataset = dataset[dataset['pickup_latitude'] < dataset['pickup_latitude'].quantile(0.999) ]

# dataset = dataset[dataset['trip_duration'] < dataset['trip_duration'].quantile(0.999)]
train = dataset

In [ ]:
plt.close()
# f, ax = plt.subplots(figsize=(12,12))

plt.scatter( range(validate.shape[0]), validate['trip_duration'])
plt.show()


In [13]:
features = list(set(list(train)) - set(['id','trip_duration', 'pickup_longitude','pickup_latitude',\
                                   'dropoff_longitude','dropoff_latitude']))

In [14]:
Train = pd.get_dummies(train[features], drop_first = True)
Validate = pd.get_dummies(validate[features], drop_first = True)

In [19]:
Validate.loc[:,'passenger_count_h'] = 0
Validate.loc[:,'passenger_count_j'] = 0


In [22]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, criterion = 'mse', max_features = 'auto', random_state = 0 )
regressor.fit(Train, Train['trip_duration'])

KeyboardInterrupt: 

In [ ]:
f, ax = plt.subplots(figsize=(12,12))
plt.hist(train['trip_duration'], bins = 1000)
plt.show()

In [ ]:
list(train)

In [ ]:

plt.close()
f, ax = plt.subplots(figsize=(12,12))
plt.scatter(range(validate.shape[0]), validate['pickup_latitude'])
plt.show()



In [ ]:
plt.close()
f, ax = plt.subplots(figsize=(12,12))
ax = plt.scatter(dataset['pickup_longitude'], dataset['pickup_latitude'])
# plt.xlim((-75, -72))
# plt.ylim((40.4, 42.3))
plt.show()

In [ ]:
plt.close()
fig, ax = plt.subplots(figsize=(15,5))
sns.boxplot(x="dropoff_longitude", y="manhattan_dist", data=dataset);
plt.show()

In [ ]:
plt.close()
fig, ax = plt.subplots(figsize=(15,5))
sns.boxplot(x="dropoff_longitude", y="manhattan_speed", data=dataset);
plt.show()

In [ ]:
plt.close()
col = 'hour'
plt.scatter(range(len(dataset[col].unique())), dataset.groupby(col).count()[['id']])
plt.show()

In [ ]:
#correlation matrix
import seaborn as sns
corrmat = train[train.select_dtypes(exclude=['object']).columns].corr()
f, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(corrmat, square=True, cmap="YlGnBu", linewidths=.5, annot=False);
plt.show()

In [ ]:
from numpy import where
where(dataset['pickup_long_round'].unique() == -74)

In [ ]:
plt.close()
colx = 'trip_duration'
# print (test[colx].unique())
# print(train.groupby(colx).count()[['Id']]/1460)
fig, ax = plt.subplots(figsize=(15,6))
plt.scatter(np.sort(train[colx])[:-4], range(train.shape[0]-4))
plt.show()
# plt.scatter(train['BsmtQual_num'], train['BsmtCond_num']);

In [ ]:
dataset['day_of_the_year'] = dataset['pickup_datetime'].dt.dayofyear

In [ ]:
list(dataset)
features  = ['id', 'vendor_id', 'pickup_time', 'day_of_the_year', 'distance', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude','dropoff_longitude',\
             'trip_duration']

In [ ]:
#converting month, day of week, day of year, time to cyclic
from numpy import cos, pi
dataset['day_of_year_cos'] = cos(((dataset['day_of_the_year'])/183)*pi)
dataset['pickup_time_cos'] = cos(((dataset['pickup_time'])/(12*3600))*pi)

In [ ]:
df = dataset[features]
df.shape

In [ ]:
#Encoding the string data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
dataset['day_week_cat'] = labelencoder_X_1.fit_transform(dataset['day_week'].values)
onehotencoder = OneHotEncoder(categorical_features = 'all', sparse = False)
W = onehotencoder.fit_transform(dataset['day_week_cat'].values.reshape(-1,1))

In [ ]:
W.shape

In [ ]:
list(df)

In [ ]:
# df is the dataset on which we are going to train
from numpy import hstack, log
list(df)
X = df.iloc[:, 0:9].values
X = hstack((X,W[:,1:]))
y = df.iloc[:, 9].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.decomposition import PCA
# 7 and 9 are latitudes
L = np.vstack((X_train[:, [5, 6]], 
               X_train[:, [7, 8]]))
pca = PCA()
L = pca.fit_transform(L)
# L_pca = pca.transform(L)
# _, (ax1, ax2) = plt.subplots(2, 1, figsize=(16,9))

# sample_ind = np.random.permutation(len(L))[:10000]

# ax1.scatter(X[sample_ind,0], X[sample_ind,1], s=1, lw=0)
# ax1.set_title('Original')

# ax2.scatter(X_pca[sample_ind,0], X_pca[sample_ind,1], s=1, lw=0)
# ax2.set_title('Rotated')

In [ ]:
X_train.shape

In [ ]:
X_train_1 = X_train[:, [5,6,7,8]]
X_test_1 = X_test[:, [5,6,7,8]]

In [ ]:
X_train[:, 5] = pca.transform(X_train_1[:, [0,1]])[:, 0] # pickup_latitude
X_train[:, 6] = pca.transform(X_train_1[:, [0,1]])[:, 1] #pickup_longitude
X_train[:, 7] = pca.transform(X_train_1[:, [2,3]])[:, 0] #dropoff_latitude
X_train[:, 8] = pca.transform(X_train_1[:, [2,3]])[:, 1] #dropoff_longitude

In [ ]:
# apply the pca transform for the test set
X_test[:, 5] = pca.transform(X_test_1[:, [0,1]])[:, 0]
X_test[:, 6] = pca.transform(X_test_1[:, [0,1]])[:, 1]
X_test[:, 7] = pca.transform(X_test_1[:, [2,3]])[:, 0]
X_test[:, 8] = pca.transform(X_test_1[:, [2,3]])[:, 1]

In [ ]:
# dataset['pickup_pca0'] = pca.transform(dataset[['pickup_latitude', 'pickup_longitude']])[:,0]
# dataset['pickup_pca1'] = pca.transform(dataset[['pickup_latitude', 'pickup_longitude']])[:,1]

# dataset['dropoff_pca0'] = pca.transform(dataset[['dropoff_latitude', 'dropoff_longitude']])[:,0]
# dataset['dropoff_pca1'] = pca.transform(dataset[['dropoff_latitude', 'dropoff_longitude']])[:,1]

In [ ]:
# # checking our target Y: trip_duration and plot
from numpy import percentile
q1 = percentile(y_train, 0.001)
q2 = percentile(y_train, 99.999)
keep = [(y_train > q1) & (y_train < q2)]

# plt.close()
# plt.scatter(range(y_train[keep].shape[0]), np.sort( y_train[keep]))
# # plt.xlabel('day')
# plt.ylabel('trip duration')
# plt.show()

In [ ]:
y_train = y_train[keep]
X_train = X_train[keep]

In [ ]:
# q1 = dataset.pickup_longitude.quantile(0.001)
# q2 = dataset.pickup_longitude.quantile(0.999)
# dataset = dataset[(dataset.pickup_longitude > q1) & (dataset.pickup_longitude < q2)]
# # checking our target Y: trip_duration and plot
q1 = percentile(X_train[:,5], 0.01, axis = 0 )
q2 = percentile(X_train[:,5], 99.99, axis = 0)
keep = [(X_train[:, 5] > q1) & (X_train[:, 5] < q2)]

X_train = X_train[keep]
y_train = y_train[keep]
# plt.close()
# plt.scatter(range(X_t.shape[0]), X_t[:,5])
# plt.xlabel('index')
# plt.ylabel('pickup latitude')
# plt.show()

In [ ]:
from numpy import percentile
q1 = percentile(X_train[:,6], 0.001, axis = 0 )
q2 = percentile(X_train[:, 6], 99.999, axis = 0)

keep = [(X_train[:, 6] > q1) & (X_train[:, 6] < q2)]

X_train= X_train[keep]
y_train = y_train[keep]
# X_t.shape
# plt.close()
# plt.scatter(range(X_t.shape[0]), X_t[:,6])
# plt.xlabel('index')
# plt.ylabel('pickup longitude')
# plt.show()

In [ ]:
q1 = percentile(X_train[:,7], 0.01, axis = 0 )
q2 = percentile(X_train[:, 7], 99.99, axis = 0)

keep = [(X_train[:, 7] > q1) & (X_train[:, 7] < q2)]

X_train= X_train[keep]
y_train= y_train[keep]
# # X_t.shape
# plt.close()
# plt.scatter(range(X_t.shape[0]), X_t[:,7])
# plt.xlabel('index')
# plt.ylabel('dropoff latitude')
# plt.show()

In [ ]:
q1 = percentile(X_train[:,8], 0.01, axis = 0 )
q2 = percentile(X_train[:, 8], 99.999, axis = 0)

keep = [(X_train[:, 8] > q1) & (X_train[:, 8] < q2)]

X_train= X_train[keep]
y_train = y_train[keep]
# # X_t.shape
# plt.close()
# plt.scatter(range(X_t.shape[0]), X_t[:,8])
# plt.xlabel('index')
# plt.ylabel('dropoff longitude')
# plt.show()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train = X_train[:, 1:]

In [ ]:
X_test = X_test[:, 1:]

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train, y = y_train)

In [ ]:
X_test = sc.transform(X_test, y_test)

In [ ]:
df_train = pd.DataFrame(X_train)
df_train.to_csv("df_train.csv")

df_test = pd.DataFrame(X_test)
df_test.to_csv('df_test.csv')

In [ ]:
y_t = (y_train-min(y_train))/(max(y_train)-min(y_train)) 

In [ ]:
y_train = log(y_train)

In [ ]:
import keras
from keras.models import Sequential, load_model #initialising the NN
from keras.layers import Dense #initialising the layers
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
# create model
model = Sequential()
model.add(Dense(units = 9, input_dim=14, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 9, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 9, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(1, kernel_initializer='uniform'))
# Compile model
adam = keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0)
model.compile(loss='mean_squared_error', optimizer=adam )

In [ ]:
# fitting the Model
model.fit(X_train, y_train, batch_size=200, epochs=50)


In [ ]:
#save the model
model.save('taxi.h5')
model.save_weights('taxi_weights.h5')

In [ ]:
# model = load_weights('taxi_weights.h5')
model = load_model('taxi.h5')
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [ ]:
from numpy import exp
y_train = exp(y_train)
y_pred_train = exp(y_pred_train)
y_pred_test = exp(y_pred_test)

In [ ]:
rmsle(y_train, y_pred_train)

In [ ]:
def rmsle_np(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
    
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

rmsle_np(y_pred_train[:10000], y_train[:10000])

In [ ]:
from numpy import array
y_train = array(y_train)

In [ ]:
import numpy as np
from math import exp
import keras.backend as K
def rmsle(y_true, y_pred):
    first_log = K.log(y_pred + 1.)
    second_log = K.log(y_true + 1.)
    return (K.mean(K.square(first_log - second_log)))**0.5



In [ ]:
# 0.26 mse ==> 0.4646 rmsle 
from numpy import vstack
y_pred = vstack((y_pred_test, y_pred_train))
y_true = hstack((y_test, y_train))
print(str(rmsle(y_train, y_pred_train)))
print(str(rmsle(y_test, y_pred_test)))

In [ ]:
from sklearn.model_selection import KFold
import numpy
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
# fix random seed for reproducibility

seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=200, batch_size=500)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=2, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Standardized: %.2f (%.2f) MSLE" % (results.mean(), results.std()))

In [ ]:
def rmsle_np(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
    
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())
rmsle_np(y_pred_train, y_test)

In [ ]:
# Adding the input layer and the hidden layer with Dropout
# number of nodes in hidden layers or the output of the input layer is taken as the avg of 11 and 1
classifier.add(Dense(units =6, init = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(p = 0.1))
classifier.add(Dense(units  =6, init = 'uniform', activation = 'relu'))
classifier.add(Dropout(p = 0.1))
# choose softmax instead of sigmoid when there are 3 possible output 
# Also output_dim = 3 in that case.
classifier.add(Dense(units =1, init = 'uniform', activation = 'sigmoid'))

#compiling the network
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

# fitting the Model
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

#save the model
classifier.save('banking.h5')
classifier.save_weights('banking_weights.h5')

#test run
classifier1 = load_model('banking.h5')
yaml_string = classifier1.to_yaml()
json_string = classifier1.to_json()

In [ ]:
X_test[1]

In [ ]:
y_pred = classifier1.predict(X_test)

In [ ]:
y_pred = (y_pred>0.5)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# new customer prediction
classifier1.predict(sc.transform(numpy.array([[0,0, 600, 1, 40, 3,  60000, 2, 1, 1, 50000]])))

In [ ]:
from keras.layers import Dropout